# QuickPBSA Integration for AG Heine
Created by Andreas Brilka Februar 2025

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import quickpbsa as pbsa
import tkinter as tk
from tkinter import filedialog
from pathlib import Path
import math

from IPython.core.display import HTML

root = tk.Tk()
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)

notebook_path = Path(os.path.abspath(""))

### Import files from ImageJ/Neurotorch Multimeasure

This cell allows you to import a file created by ImageJ/Neurotorch by measuring ROIs

In [ ]:
# Execute this cell to open a file
tempFile = None
if not (p := filedialog.askopenfilename(parent=root, title="Open a multi measure CSV file", defaultextension="*.csv", filetypes=[(".csv", "*.csv"), ("All files", "*.*")])) or not (path := Path(p)).exists():
    print("Your provided path is invalid or empty")
else:
    tempFile = notebook_path / "file.csv"
    data_multimeasure = pd.read_csv(path, sep=",", header=0, index_col=0)
    display(HTML(f"<h3> {path.name} </h3>"))
    display(data_multimeasure)

    data_multimeasure.reset_index(drop=True, inplace=True)
    data = data_multimeasure.transpose().copy()
    data["id"] = data.index
    data = data[["id"] + list(data.columns[0:-1])]
    data.to_csv(tempFile, index=False)

    nrows, ncols = math.ceil(len(data_multimeasure.columns)/3), 3
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 5*nrows))
    axes = axes.flatten()
    for ax in axes:
        ax.set_axis_off()
    for i, df_c in enumerate(data_multimeasure.columns):
        ax = axes[i]
        ax.set_axis_on()
        x, y = data_multimeasure.index, data_multimeasure[df_c]
        ax.plot(x,y)
        ax.set_xlabel("Frame")
        ax.set_title(f"{df_c}")
    fig.tight_layout()
    plt.show()

### Detect steps by using quickPBSA
Change here the parameters for detection!

In [ ]:
KVthreshold = 20 # KV threshold (1/3 to 1/2 of typical last step)
KV_maxiter = 100   # Maximum KV iterations (much higher than expected number of steps)
kvout, kv_outfile, kv_jsonfile = pbsa.steps_preliminary.kv_file(str(tempFile), KVthreshold, KV_maxiter)

In [ ]:
# Plot output
nrows, ncols = math.ceil((len(kvout) // 5)/2), 2
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 5*nrows))
axes = axes.flatten()
for ax in axes:
    ax.set_axis_off()
for i in range(len(kvout)//5):
    ax = axes[i]
    ax.set_axis_on()
    name = kvout["id"][5*i]
    x = range(len(kvout.columns[10:]))
    y = np.array(kvout.iloc[5*i, 10:])
    y_mean = np.array(kvout.iloc[5*i + 1, 10:])
    y_std = np.array(kvout.iloc[5*i + 2, 10:])
    y_fintens = np.array(kvout.iloc[5*i + 3, 10:])
    y_fkv = np.array(kvout.iloc[5*i + 4, 10:])
    ax.plot(x,y, label="Trace")
    ax.plot(x,y_mean, label="Means")
    ax.plot(x,y_std, label="Standard Deviations")
    ax.plot(x,y_fintens, label="Fluorophores Ruler")
    ax.plot(x,y_fkv, label="Fluorophores Preliminary")
    ax.set_xlabel("Frame")
    ax.set_title(f"{name}")
    ax.legend()
fig.tight_layout()
plt.show()